In [2]:
import sklearn.datasets as datasets            ##all imports used in the code
import gensim.utils as gensimUtils
import nltk 
from gensim import corpora
from gensim import models
import pandas as pd
import scipy.spatial.distance as scipyDistance
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import gensim.parsing.preprocessing as preprocessing
from gensim.models import Phrases
import numpy as np
from nltk.corpus import stopwords
import re
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import SnowballStemmer
from nltk.metrics import scores
from sklearn.metrics import confusion_matrix
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.metrics import cohen_kappa_score
from easy_table import EasyTable
import sys

sys.argv = [""]

NAVY=pd.read_csv('CleanedNAVYscoresclean.csv', encoding='latin1') #get data from NAVY response scoring
df = pd.DataFrame(NAVY)    #set NAVY as dataframe



In [3]:
df.head()

,J1,J2,J3,LSA,Regex,Match,Question,Type,LSA_Regex
0,2,2,2,0.75,0.27,0,Main,Good,0.39
1,3,3,4,4.00,0.27,0,Main,Good,1.20
2,2,3,2,0.01,0.00,0,Main,Good,0.00
3,4,4,3,0.79,0.00,0,Main,Good,0.20
4,4,4,2,0.73,0.33,0,Main,Good,0.43


In [10]:
Jthreshv = 5
J2threshv = 5
J3threshv = 5
LSAthreshv = .889   #Stringent(S) thresholds 
RegExthreshv = .8

JthreshIv = 4
J2threshIv = 4
J3threshIv = 4
LSAthreshIv = .669  #Intermediate(I) thresholds 
RegExthreshvI = .66

JthreshLv = 3
J2threshLv = 3
J3threshLv = 3
LSAthreshLv = .66 #Lenient(L) thresholds
RegExthreshvL = .4

Jthresh = []                        ##the next for loops are about coding match values for human judges and computer models(LSA,w2v,w2vB,D2V)
for value in NAVY.J1: 
    if value == Jthreshv: 
        Jthresh.append(1)  
    else: 
        Jthresh.append(0) 
J2thresh = [] 
for value in NAVY.J2: 
    if value == J2threshv: 
        J2thresh.append(1)  
    else: 
        J2thresh.append(0)    
J3thresh = [] 
for value in NAVY.J3: 
    if value == J3threshv: 
        J3thresh.append(1)  
    else: 
        J3thresh.append(0)   
LSAthresh = [] 
for value in df['LSA']: 
    if value >= LSAthreshv: 
        LSAthresh.append(1)
    else: 
        LSAthresh.append(0)
RegExthresh = [] 
for value in df['Regex']: 
    if value >= RegExthreshv: 
        RegExthresh.append(1)
    else: 
        RegExthresh.append(0)


     
JthreshI = []                        ##the next for loops are about coding match values for human judges and computer models in intermediate thresholds.
for value in NAVY.J1: 
    if value >= JthreshIv: 
        JthreshI.append(1)  
    else: 
        JthreshI.append(0) 
J2threshI = [] 
for value in NAVY.J2: 
    if value >= J2threshIv: 
        J2threshI.append(1)  
    else: 
        J2threshI.append(0)
J3threshI = [] 
for value in NAVY.J3: 
    if value >= J3threshIv: 
        J3threshI.append(1)  
    else: 
        J3threshI.append(0)
LSAthreshI = [] 
for value in df['LSA']: 
    if value >= LSAthreshIv: 
        LSAthreshI.append(1)
    else: 
        LSAthreshI.append(0)
RegExthreshI= []  
for value in df['Regex']: 
    if value >= RegExthreshvI: 
        RegExthreshI.append(1)
    else: 
        RegExthreshI.append(0)

JthreshL = []                        ##the next 8 for loops are about coding match values for human judges and computer models in lenient thresholds.
for value in NAVY.J1: 
    if value >= JthreshLv: 
        JthreshL.append(1)  
    else: 
        JthreshL.append(0) 
J2threshL = [] 
for value in NAVY.J2: 
    if value >= J2threshLv: 
        J2threshL.append(1)  
    else: 
        J2threshL.append(0)  
J3threshL = [] 
for value in NAVY.J3: 
    if value >= J3threshLv: 
        J3threshL.append(1)  
    else: 
        J3threshL.append(0)         
LSAthreshL = [] 
for value in df['LSA']: 
    if value >= LSAthreshLv: 
        LSAthreshL.append(1)
    else: 
        LSAthreshL.append(0)  
RegExthreshL = []
for value in df['Regex']: 
    if value >= RegExthreshvL: 
        RegExthreshL.append(1)
    else: 
        RegExthreshL.append(0)
        
LSAcm = confusion_matrix(Jthresh, LSAthresh)  #confusion matrix to plug into precision and recall tool
LSAcm2 = confusion_matrix(J2thresh, LSAthresh)
LSAcm3 = confusion_matrix(J3thresh, LSAthresh)   #confusion matrix to plug into precision and recall tool
LSAcmJ12 = confusion_matrix(Jthresh, J2thresh)
LSAcmJ13 = confusion_matrix(Jthresh, J3thresh)
LSAcmJ23 = confusion_matrix(J2thresh, J3thresh)

LSAcmI = confusion_matrix(JthreshI, LSAthreshI)  #confusion matrix to plug into precision and recall tool
LSAcmI2 = confusion_matrix(J2threshI, LSAthreshI)
LSAcmI3 = confusion_matrix(J3threshI, LSAthreshI)  #confusion matrix to plug into precision and recall tool
LSAcmJI12 = confusion_matrix(JthreshI, J2threshI) 
LSAcmJI13 = confusion_matrix(JthreshI, J3threshI) 
LSAcmJI23 = confusion_matrix(J2threshI, J3threshI) 

LSAcmL = confusion_matrix(JthreshL, LSAthreshL)  #confusion matrix to plug into precision and recall tool
LSAcmL2 = confusion_matrix(J2threshL, LSAthreshL)
LSAcmL3 = confusion_matrix(J3threshL, LSAthreshL)  #confusion matrix to plug into precision and recall tool
LSAcmJL12 = confusion_matrix(JthreshL, J2threshL)
LSAcmJL13 = confusion_matrix(JthreshL, J3threshL)
LSAcmJL23 = confusion_matrix(J2threshL, J3threshL)


tpJ12 = LSAcmJ12[1,1]
tnJ12 = LSAcmJ12[0,0]
pJ12 = (LSAcmJ12[1,0] + LSAcmJ12[1,1])                    ##Judge 1 vs Judge 2 stringent
nJ12 = (LSAcmJ12[0,0] + LSAcmJ12[0,1])
fpJ12 = LSAcmJ12[1,0]
fnJ12 = LSAcmJ12[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyJ12 = (tpJ12 + tnJ12) / (pJ12 + nJ12)
# precision tp / (tp + fp)
precisionJ12 = tpJ12 / (tpJ12 + fpJ12)
# recall: tp / (tp + fn)

recallJ12 = tpJ12 / (tpJ12 + fnJ12)
# f1: 2 tp / (2 tp + fp + fn)
f1J12 = 2*tpJ12 / (2*tpJ12 + fpJ12 + fnJ12)

tpJ13 = LSAcmJ13[1,1]
tnJ13 = LSAcmJ13[0,0]
pJ13 = (LSAcmJ13[1,0] + LSAcmJ13[1,1])                    ##Judge vs Judge 2 stringent
nJ13 = (LSAcmJ13[0,0] + LSAcmJ13[0,1])
fpJ13 = LSAcmJ13[1,0]
fnJ13 = LSAcmJ13[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyJ13 = (tpJ13 + tnJ13) / (pJ13 + nJ13)
# precision tp / (tp + fp)
precisionJ13 = tpJ13 / (tpJ13 + fpJ13)
# recall: tp / (tp + fn)

recallJ13 = tpJ13 / (tpJ13 + fnJ13)
# f1: 2 tp / (2 tp + fp + fn)
f1J13 = 2*tpJ13 / (2*tpJ13 + fpJ13 + fnJ13)

tpJ23 = LSAcmJ23[1,1]
tnJ23 = LSAcmJ23[0,0]
pJ23 = (LSAcmJ23[1,0] + LSAcmJ23[1,1])                    ##Judge vs Judge 2 stringent
nJ23 = (LSAcmJ23[0,0] + LSAcmJ23[0,1])
fpJ23 = LSAcmJ23[1,0]
fnJ23 = LSAcmJ23[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyJ23 = (tpJ23 + tnJ23) / (pJ23 + nJ23)
# precision tp / (tp + fp)
precisionJ23 = tpJ23 / (tpJ23 + fpJ23)
# recall: tp / (tp + fn)

recallJ23 = tpJ23 / (tpJ23 + fnJ23)
# f1: 2 tp / (2 tp + fp + fn)
f1J23 = 2*tpJ23 / (2*tpJ23 + fpJ23 + fnJ23)

tp = LSAcm[1,1]
tn = LSAcm[0,0]
p = (LSAcm[1,0] + LSAcm[1,1])                    ##LSA vs Judge 1
n = (LSAcm[0,0] + LSAcm[0,1])
fp = LSAcm[1,0]
fn = LSAcm[0,1]
# accuracy: (tp + tn) / (p + n)
accuracy = (tp + tn) / (p + n)
# precision tp / (tp + fp)
precision = tp / (tp + fp)
# recall: tp / (tp + fn)
recall = tp / (tp + fn)
# f1: 2 tp / (2 tp + fp + fn)
f1 = 2*tp / (2*tp + fp + fn)

tp2 = LSAcm2[1,1]
tn2 = LSAcm2[0,0]
p2 = (LSAcm2[1,0] + LSAcm2[1,1])                    ##LSA vs Judge 2
n2 = (LSAcm2[0,0] + LSAcm2[0,1])
fp2 = LSAcm2[1,0]
fn2 = LSAcm2[0,1]
# accuracy: (tp + tn) / (p + n)
accuracy2 = (tp2 + tn2) / (p2 + n2)
# precision tp / (tp + fp)
precision2 = tp2 / (tp2 + fp2)
# recall: tp / (tp + fn)
recall2 = tp2 / (tp2 + fn2)
# f1: 2 tp / (2 tp + fp + fn)
f12 = 2*tp2 / (2*tp2 + fp2 + fn2)

tp3 = LSAcm3[1,1]
tn3 = LSAcm3[0,0]
p3 = (LSAcm3[1,0] + LSAcm3[1,1])                    ##LSA vs Judge 3
n3 = (LSAcm3[0,0] + LSAcm3[0,1])
fp3 = LSAcm3[1,0]
fn3 = LSAcm3[0,1]
# accuracy: (tp + tn) / (p + n)
accuracy3 = (tp3 + tn3) / (p3 + n3)
# precision tp / (tp + fp)
precision3 = tp3 / (tp3 + fp3)
# recall: tp / (tp + fn)
recall3 = tp3 / (tp3 + fn3)
# f1: 2 tp / (2 tp + fp + fn)
f13 = 2*tp3 / (2*tp3 + fp3 + fn3)



tpJI12 = LSAcmJI12[1,1]
tnJI12 = LSAcmJI12[0,0]
pJI12 = (LSAcmJI12[1,0] + LSAcmJI12[1,1])
nJI12 = (LSAcmJI12[0,0] + LSAcmJI12[0,1])
fpJI12 = LSAcmJI12[1,0]
fnJI12 = LSAcmJI12[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyJI12 = (tpJI12 + tnJI12) / (pJI12 + nJI12)
# precision tp / (tp + fp)
precisionJI12 = tpJI12 / (tpJI12 + fpJI12)
# recall: tp / (tp + fn)
recallJI12 = tpJI12 / (tpJI12 + fnJI12)
# f1: 2 tp / (2 tp + fp + fn)
f1JI12 = 2*tpJI12 / (2*tpJI12 + fpJI12 + fnJI12)

tpJI13 = LSAcmJI13[1,1]
tnJI13 = LSAcmJI13[0,0]
pJI13 = (LSAcmJI13[1,0] + LSAcmJI13[1,1])
nJI13 = (LSAcmJI13[0,0] + LSAcmJI13[0,1])
fpJI13 = LSAcmJI13[1,0]
fnJI13 = LSAcmJI13[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyJI13 = (tpJI13 + tnJI13) / (pJI13 + nJI13)
# precision tp / (tp + fp)
precisionJI13 = tpJI13 / (tpJI13 + fpJI13)
# recall: tp / (tp + fn)
recallJI13 = tpJI13 / (tpJI13 + fnJI13)
# f1: 2 tp / (2 tp + fp + fn)
f1JI13 = 2*tpJI13 / (2*tpJI13 + fpJI13 + fnJI13)

tpJI23 = LSAcmJI23[1,1]
tnJI23 = LSAcmJI23[0,0]
pJI23 = (LSAcmJI23[1,0] + LSAcmJI23[1,1])
nJI23 = (LSAcmJI23[0,0] + LSAcmJI23[0,1])
fpJI23 = LSAcmJI23[1,0]
fnJI23 = LSAcmJI23[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyJI23 = (tpJI23 + tnJI23) / (pJI23 + nJI23)
# precision tp / (tp + fp)
precisionJI23 = tpJI23 / (tpJI23 + fpJI23)
# recall: tp / (tp + fn)
recallJI23 = tpJI23 / (tpJI23 + fnJI23)
# f1: 2 tp / (2 tp + fp + fn)
f1JI23 = 2*tpJI23 / (2*tpJI23 + fpJI23 + fnJI23)

tpI = LSAcmI[1,1]
tnI = LSAcmI[0,0]
pI = (LSAcmI[1,0] + LSAcmI[1,1])
nI = (LSAcmI[0,0] + LSAcmI[0,1])
fpI = LSAcmI[1,0]
fnI = LSAcmI[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyI = (tpI + tnI) / (pI + nI)
# precision tp / (tp + fp)
precisionI = tpI / (tpI + fpI)
# recall: tp / (tp + fn)
recallI = tpI / (tpI + fnI)
# f1: 2 tp / (2 tp + fp + fn)
f1I = 2*tpI / (2*tpI + fpI + fnI)

tpI2 = LSAcmI2[1,1]
tnI2 = LSAcmI2[0,0]
pI2 = (LSAcmI2[1,0] + LSAcmI2[1,1])
nI2 = (LSAcmI2[0,0] + LSAcmI2[0,1])
fpI2 = LSAcmI2[1,0]
fnI2 = LSAcmI2[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyI2 = (tpI2 + tnI2) / (pI2 + nI2)
# precision tp / (tp + fp)
precisionI2 = tpI2 / (tpI2 + fpI2)
# recall: tp / (tp + fn)
recallI2 = tpI2 / (tpI2 + fnI2)
# f1: 2 tp / (2 tp + fp + fn)
f1I2 = 2*tpI2 / (2*tpI2 + fpI2 + fnI2)

tpI3 = LSAcmI3[1,1]
tnI3 = LSAcmI3[0,0]
pI3 = (LSAcmI3[1,0] + LSAcmI3[1,1])
nI3 = (LSAcmI3[0,0] + LSAcmI3[0,1])
fpI3 = LSAcmI3[1,0]
fnI3 = LSAcmI3[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyI3 = (tpI3 + tnI3) / (pI3 + nI3)
# precision tp / (tp + fp)
precisionI3 = tpI3 / (tpI3 + fpI3)
# recall: tp / (tp + fn)
recallI3 = tpI3 / (tpI3 + fnI3)
# f1: 2 tp / (2 tp + fp + fn)
f1I3 = 2*tpI3 / (2*tpI3 + fpI3 + fnI3)


tpJL12 = LSAcmJL12[1,1]
tnJL12 = LSAcmJL12[0,0]
pJL12 = (LSAcmJL12[1,0] + LSAcmJL12[1,1])
nJL12 = (LSAcmJL12[0,0] + LSAcmJL12[0,1])
fpJL12 = LSAcmJL12[1,0]
fnJL12 = LSAcmJL12[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyJL12 = (tpJL12 + tnJL12) / (pJL12 + nJL12)
# precision tp / (tp + fp)
precisionJL12 = tpJL12 / (tpJL12 + fpJL12)
recallJL12 = tpJL12 / (tpJL12 + fnJL12)
# f1: 2 tp / (2 tp + fp + fn)
f1JL12 = 2*tpJL12 / (2*tpJL12 + fpJL12 + fnJL12)

tpJL13 = LSAcmJL13[1,1]
tnJL13 = LSAcmJL13[0,0]
pJL13 = (LSAcmJL13[1,0] + LSAcmJL13[1,1])
nJL13 = (LSAcmJL13[0,0] + LSAcmJL13[0,1])
fpJL13 = LSAcmJL13[1,0]
fnJL13 = LSAcmJL13[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyJL13 = (tpJL13 + tnJL13) / (pJL13 + nJL13)
# precision tp / (tp + fp)
precisionJL13 = tpJL13 / (tpJL13 + fpJL13)
recallJL13 = tpJL13 / (tpJL13 + fnJL13)
# f1: 2 tp / (2 tp + fp + fn)
f1JL13 = 2*tpJL13 / (2*tpJL13 + fpJL13 + fnJL13)

tpJL23 = LSAcmJL23[1,1]
tnJL23 = LSAcmJL23[0,0]
pJL23 = (LSAcmJL23[1,0] + LSAcmJL23[1,1])
nJL23 = (LSAcmJL23[0,0] + LSAcmJL23[0,1])
fpJL23 = LSAcmJL23[1,0]
fnJL23 = LSAcmJL23[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyJL23 = (tpJL23 + tnJL23) / (pJL23 + nJL23)
# precision tp / (tp + fp)
precisionJL23 = tpJL23 / (tpJL23 + fpJL23)
recallJL23 = tpJL23 / (tpJL23 + fnJL23)
# f1: 2 tp / (2 tp + fp + fn)
f1JL23 = 2*tpJL23 / (2*tpJL23 + fpJL23 + fnJL23)

tpL = LSAcmL[1,1]
tnL = LSAcmL[0,0]
pL = (LSAcmL[1,0] + LSAcmL[1,1])
nL = (LSAcmL[0,0] + LSAcmL[0,1])
fpL = LSAcmL[1,0]
fnL = LSAcmL[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyL = (tpL + tnL) / (pL + nL)
# precision tp / (tp + fp)
precisionL = tpL / (tpL + fpL)
recallL = tpL / (tpL + fnL)
# f1: 2 tp / (2 tp + fp + fn)
f1L = 2*tpL / (2*tpL + fpL + fnL)

tpL2 = LSAcmL2[1,1]
tnL2 = LSAcmL2[0,0]
pL2 = (LSAcmL2[1,0] + LSAcmL2[1,1])
nL2 = (LSAcmL2[0,0] + LSAcmL2[0,1])
fpL2 = LSAcmL2[1,0]
fnL2 = LSAcmL2[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyL2 = (tpL2 + tnL2) / (pL2 + nL2)
# precision tp / (tp + fp)
precisionL2 = tpL2 / (tpL2 + fpL2)
recallL2 = tpL2 / (tpL2 + fnL2)
# f1: 2 tp / (2 tp + fp + fn)
f1L2 = 2*tpL2 / (2*tpL2 + fpL2 + fnL2)

tpL3 = LSAcmL3[1,1]
tnL3 = LSAcmL3[0,0]
pL3 = (LSAcmL3[1,0] + LSAcmL3[1,1])
nL3 = (LSAcmL3[0,0] + LSAcmL3[0,1])
fpL3 = LSAcmL3[1,0]
fnL3 = LSAcmL3[0,1]
# accuracy: (tp + tn) / (p + n)
accuracyL3 = (tpL3 + tnL3) / (pL3 + nL3)
# precision tp / (tp + fp)
precisionL3 = tpL3 / (tpL3 + fpL3)
recallL3 = tpL3 / (tpL3 + fnL3)
# f1: 2 tp / (2 tp + fp + fn)
f1L3 = 2*tpL3 / (2*tpL3 + fpL3 + fnL3)

RegExcm = confusion_matrix(Jthresh, RegExthresh)  #confusion matrix to plug into precision and recall tool
RegExcm2 = confusion_matrix(J2thresh, RegExthresh)
RegExcm3 = confusion_matrix(J3thresh, RegExthresh)  #confusion matrix to plug into precision and recall tool
RegExcmJ12 = confusion_matrix(Jthresh, J2thresh)
RegExcmJ13 = confusion_matrix(Jthresh, J3thresh)
RegExcmJ23 = confusion_matrix(J2thresh, J3thresh)

RegExcmI = confusion_matrix(JthreshI, RegExthreshI)  #confusion matrix to plug into precision and recall tool
RegExcmI2 = confusion_matrix(J2threshI, RegExthreshI)
RegExcmI3 = confusion_matrix(J3threshI, RegExthreshI)  #confusion matrix to plug into precision and recall tool
RegExcmJI12 = confusion_matrix(JthreshI, J2threshI) 
RegExcmJI13 = confusion_matrix(JthreshI, J3threshI)
RegExcmJI23 = confusion_matrix(J2threshI, J3threshI)

RegExcmL = confusion_matrix(JthreshL, RegExthreshL)  #confusion matrix to plug into precision and recall tool
RegExcmL2 = confusion_matrix(J2threshL, RegExthreshL)
RegExcmL3 = confusion_matrix(J3threshL, RegExthreshL)   #confusion matrix to plug into precision and recall tool
RegExcmJL12 = confusion_matrix(JthreshL, J2threshL)
RegExcmJL13 = confusion_matrix(JthreshL, J3threshL)
RegExcmJL23 = confusion_matrix(J2threshL, J3threshL)



REtpJ12 = RegExcmJ12[1,1]
REtnJ12 = RegExcmJ12[0,0]
REpJ12 = (RegExcmJ12[1,0] + RegExcmJ12[1,1])                    ##J12udge vs J12udge 2 stringent
REnJ12 = (RegExcmJ12[0,0] + RegExcmJ12[0,1])
REfpJ12 = RegExcmJ12[1,0]
REfnJ12 = RegExcmJ12[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyJ12 = (REtpJ12 + REtnJ12) / (REpJ12 + REnJ12)
# precision tp / (tp + fp)
REprecisionJ12 = REtpJ12 / (REtpJ12 + REfpJ12)
# recall: tp / (tp + fn)

RErecallJ12 = REtpJ12 / (REtpJ12 + REfnJ12)
# f1: 2 tp / (2 tp + fp + fn)
REf1J12 = 2*REtpJ12 / (2*REtpJ12 + REfpJ12 + REfnJ12)

REtpJ13 = RegExcmJ13[1,1]
REtnJ13 = RegExcmJ13[0,0]
REpJ13 = (RegExcmJ13[1,0] + RegExcmJ13[1,1])                    ##J13udge vs J13udge 2 stringent
REnJ13 = (RegExcmJ13[0,0] + RegExcmJ13[0,1])
REfpJ13 = RegExcmJ13[1,0]
REfnJ13 = RegExcmJ13[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyJ13 = (REtpJ13 + REtnJ13) / (REpJ13 + REnJ13)
# precision tp / (tp + fp)
REprecisionJ13 = REtpJ13 / (REtpJ13 + REfpJ13)
# recall: tp / (tp + fn)

RErecallJ13 = REtpJ13 / (REtpJ13 + REfnJ13)
# f1: 2 tp / (2 tp + fp + fn)
REf1J13 = 2*REtpJ13 / (2*REtpJ13 + REfpJ13 + REfnJ13)

REtpJ23 = RegExcmJ23[1,1]
REtnJ23 = RegExcmJ23[0,0]
REpJ23 = (RegExcmJ23[1,0] + RegExcmJ23[1,1])                    ##J23udge vs J23udge 2 stringent
REnJ23 = (RegExcmJ23[0,0] + RegExcmJ23[0,1])
REfpJ23 = RegExcmJ23[1,0]
REfnJ23 = RegExcmJ23[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyJ23 = (REtpJ23 + REtnJ23) / (REpJ23 + REnJ23)
# precision tp / (tp + fp)
REprecisionJ23 = REtpJ23 / (REtpJ23 + REfpJ23)
# recall: tp / (tp + fn)

RErecallJ23 = REtpJ23 / (REtpJ23 + REfnJ23)
# f1: 2 tp / (2 tp + fp + fn)
REf1J23 = 2*REtpJ23 / (2*REtpJ23 + REfpJ23 + REfnJ23)

REtp = RegExcm[1,1]
REtn = RegExcm[0,0]
REp = (RegExcm[1,0] + RegExcm[1,1])                    ##RegEx vs Judge 1
REn = (RegExcm[0,0] + RegExcm[0,1])
REfp = RegExcm[1,0]
REfn = RegExcm[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracy = (REtp + REtn) / (REp + REn)
# precision tp / (tp + fp)
REprecision = REtp / (REtp + REfp)
# recall: tp / (tp + fn)
RErecall = REtp / (REtp + REfn)
# f1: 2 tp / (2 tp + fp + fn)
REf1 = 2*REtp / (2*REtp + REfp + REfn)

REtp2 = RegExcm2[1,1]
REtn2 = RegExcm2[0,0]
REp2 = (RegExcm2[1,0] + RegExcm2[1,1])                    ##RegEx vs Judge 2
REn2 = (RegExcm2[0,0] + RegExcm2[0,1])
REfp2 = RegExcm2[1,0]
REfn2 = RegExcm2[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracy2 = (REtp2 + REtn2) / (REp2 + REn2)
# precision tp / (tp + fp)
REprecision2 = REtp2 / (REtp2 + REfp2)
# recall: tp / (tp + fn)
RErecall2 = REtp2 / (REtp2 + REfn2)
# f1: 2 tp / (2 tp + fp + fn)
REf12 = 2*REtp2 / (2*REtp2 + REfp2 + REfn2)

REtp3 = RegExcm3[1,1]
REtn3 = RegExcm3[0,0]
REp3 = (RegExcm3[1,0] + RegExcm3[1,1])                    ##RegEx vs Judge 3
REn3 = (RegExcm3[0,0] + RegExcm3[0,1])
REfp3 = RegExcm3[1,0]
REfn3 = RegExcm3[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracy3 = (REtp3 + REtn3) / (REp3 + REn3)
# precision tp / (tp + fp)
REprecision3 = REtp3 / (REtp3 + REfp3)
# recall: tp / (tp + fn)
RErecall3 = REtp3 / (REtp3 + REfn3)
# f1: 2 tp / (2 tp + fp + fn)
REf13 = 2*REtp3 / (2*REtp3 + REfp3 + REfn3)


REtpJI12 = RegExcmJI12[1,1]
REtnJI12 = RegExcmJI12[0,0]
REpJI12 = (RegExcmJI12[1,0] + RegExcmJI12[1,1])
REnJI12 = (RegExcmJI12[0,0] + RegExcmJI12[0,1])
REfpJI12 = RegExcmJI12[1,0]
REfnJI12 = RegExcmJI12[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyJI12 = (REtpJI12 + REtnJI12) / (REpJI12 + REnJI12)
# precision tp / (tp + fp)
REprecisionJI12 = REtpJI12 / (REtpJI12 + REfpJI12)
# recall: tp / (tp + fn)
RErecallJI12 = REtpJI12 / (REtpJI12 + REfnJI12)
# f1: 2 tp / (2 tp + fp + fn)
REf1JI12 = 2*REtpJI12 / (2*REtpJI12 + REfpJI12 + REfnJI12)

REtpJI13 = RegExcmJI13[1,1]
REtnJI13 = RegExcmJI13[0,0]
REpJI13 = (RegExcmJI13[1,0] + RegExcmJI13[1,1])
REnJI13 = (RegExcmJI13[0,0] + RegExcmJI13[0,1])
REfpJI13 = RegExcmJI13[1,0]
REfnJI13 = RegExcmJI13[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyJI13 = (REtpJI13 + REtnJI13) / (REpJI13 + REnJI13)
# precision tp / (tp + fp)
REprecisionJI13 = REtpJI13 / (REtpJI13 + REfpJI13)
# recall: tp / (tp + fn)
RErecallJI13 = REtpJI13 / (REtpJI13 + REfnJI13)
# f1: 2 tp / (2 tp + fp + fn)
REf1JI13 = 2*REtpJI13 / (2*REtpJI13 + REfpJI13 + REfnJI13)

REtpJI23 = RegExcmJI23[1,1]
REtnJI23 = RegExcmJI23[0,0]
REpJI23 = (RegExcmJI23[1,0] + RegExcmJI23[1,1])
REnJI23 = (RegExcmJI23[0,0] + RegExcmJI23[0,1])
REfpJI23 = RegExcmJI23[1,0]
REfnJI23 = RegExcmJI23[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyJI23 = (REtpJI23 + REtnJI23) / (REpJI23 + REnJI23)
# precision tp / (tp + fp)
REprecisionJI23 = REtpJI23 / (REtpJI23 + REfpJI23)
# recall: tp / (tp + fn)
RErecallJI23 = REtpJI23 / (REtpJI23 + REfnJI23)
# f1: 2 tp / (2 tp + fp + fn)
REf1JI23 = 2*REtpJI23 / (2*REtpJI23 + REfpJI23 + REfnJI23)

REtpI = RegExcmI[1,1]
REtnI = RegExcmI[0,0]
REpI = (RegExcmI[1,0] + RegExcmI[1,1])
REnI = (RegExcmI[0,0] + RegExcmI[0,1])
REfpI = RegExcmI[1,0]
REfnI = RegExcmI[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyI = (REtpI + REtnI) / (REpI + REnI)
# precision tp / (tp + fp)
REprecisionI = REtpI / (REtpI + REfpI)
# recall: tp / (tp + fn)
RErecallI = REtpI / (REtpI + REfnI)
# f1: 2 tp / (2 tp + fp + fn)
REf1I = 2*REtpI / (2*REtpI + REfpI + REfnI)

REtpI2 = RegExcmI2[1,1]
REtnI2 = RegExcmI2[0,0]
REpI2 = (RegExcmI2[1,0] + RegExcmI2[1,1])
REnI2 = (RegExcmI2[0,0] + RegExcmI2[0,1])
REfpI2 = RegExcmI2[1,0]
REfnI2 = RegExcmI2[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyI2 = (REtpI2 + REtnI2) / (REpI2 + REnI2)
# precision tp / (tp + fp)
REprecisionI2 = REtpI2 / (REtpI2 + REfpI2)
# recall: tp / (tp + fn)
RErecallI2 = REtpI2 / (REtpI2 + REfnI2)
# f1: 2 tp / (2 tp + fp + fn)
REf1I2 = 2*REtpI2 / (2*REtpI2 + REfpI2 + REfnI2)

REtpI3 = RegExcmI3[1,1]
REtnI3 = RegExcmI3[0,0]
REpI3 = (RegExcmI3[1,0] + RegExcmI3[1,1])
REnI3 = (RegExcmI3[0,0] + RegExcmI3[0,1])
REfpI3 = RegExcmI3[1,0]
REfnI3 = RegExcmI3[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyI3 = (REtpI3 + REtnI3) / (REpI3 + REnI3)
# precision tp / (tp + fp)
REprecisionI3 = REtpI3 / (REtpI3 + REfpI3)
# recall: tp / (tp + fn)
RErecallI3 = REtpI3 / (REtpI3 + REfnI3)
# f1: 2 tp / (2 tp + fp + fn)
REf1I3 = 2*REtpI3 / (2*REtpI3 + REfpI3 + REfnI3)


REtpJL12 = RegExcmJL12[1,1]
REtnJL12 = RegExcmJL12[0,0]
REpJL12 = (RegExcmJL12[1,0] + RegExcmJL12[1,1])
REnJL12 = (RegExcmJL12[0,0] + RegExcmJL12[0,1])
REfpJL12 = RegExcmJL12[1,0]
REfnJL12 = RegExcmJL12[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyJL12 = (REtpJL12 + REtnJL12) / (REpJL12 + REnJL12)
# precision tp / (tp + fp)
REprecisionJL12 = REtpJL12 / (REtpJL12 + REfpJL12)
RErecallJL12 = REtpJL12 / (REtpJL12 + REfnJL12)
# f1: 2 tp / (2 tp + fp + fn)
REf1JL12 = 2*REtpJL12 / (2*REtpJL12 + REfpJL12 + REfnJL12)

REtpJL13 = RegExcmJL13[1,1]
REtnJL13 = RegExcmJL13[0,0]
REpJL13 = (RegExcmJL13[1,0] + RegExcmJL13[1,1])
REnJL13 = (RegExcmJL13[0,0] + RegExcmJL13[0,1])
REfpJL13 = RegExcmJL13[1,0]
REfnJL13 = RegExcmJL13[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyJL13 = (REtpJL13 + REtnJL13) / (REpJL13 + REnJL13)
# precision tp / (tp + fp)
REprecisionJL13 = REtpJL13 / (REtpJL13 + REfpJL13)
RErecallJL13 = REtpJL13 / (REtpJL13 + REfnJL13)
# f1: 2 tp / (2 tp + fp + fn)
REf1JL13 = 2*REtpJL13 / (2*REtpJL13 + REfpJL13 + REfnJL13)

REtpJL23 = RegExcmJL23[1,1]
REtnJL23 = RegExcmJL23[0,0]
REpJL23 = (RegExcmJL23[1,0] + RegExcmJL23[1,1])
REnJL23 = (RegExcmJL23[0,0] + RegExcmJL23[0,1])
REfpJL23 = RegExcmJL23[1,0]
REfnJL23 = RegExcmJL23[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyJL23 = (REtpJL23 + REtnJL23) / (REpJL23 + REnJL23)
# precision tp / (tp + fp)
REprecisionJL23 = REtpJL23 / (REtpJL23 + REfpJL23)
RErecallJL23 = REtpJL23 / (REtpJL23 + REfnJL23)
# f1: 2 tp / (2 tp + fp + fn)
REf1JL23 = 2*REtpJL23 / (2*REtpJL23 + REfpJL23 + REfnJL23)

REtpL = RegExcmL[1,1]
REtnL = RegExcmL[0,0]
REpL = (RegExcmL[1,0] + RegExcmL[1,1])
REnL = (RegExcmL[0,0] + RegExcmL[0,1])
REfpL = RegExcmL[1,0]
REfnL = RegExcmL[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyL = (REtpL + REtnL) / (REpL + REnL)
# precision tp / (tp + fp)
REprecisionL = REtpL / (REtpL + REfpL)
RErecallL = REtpL / (REtpL + REfnL)
# f1: 2 tp / (2 tp + fp + fn)
REf1L = 2*REtpL / (2*REtpL + REfpL + REfnL)

REtpL2 = RegExcmL2[1,1]
REtnL2 = RegExcmL2[0,0]
REpL2 = (RegExcmL2[1,0] + RegExcmL2[1,1])
REnL2 = (RegExcmL2[0,0] + RegExcmL2[0,1])
REfpL2 = RegExcmL2[1,0]
REfnL2 = RegExcmL2[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyL2 = (REtpL2 + REtnL2) / (REpL2 + REnL2)
# precision tp / (tp + fp)
REprecisionL2 = REtpL2 / (REtpL2 + REfpL2)
RErecallL2 = REtpL2 / (REtpL2 + REfnL2)
# f1: 2 tp / (2 tp + fp + fn)
REf1L2 = 2*REtpL2 / (2*REtpL2 + REfpL2 + REfnL2)

REtpL3 = RegExcmL3[1,1]
REtnL3 = RegExcmL3[0,0]
REpL3 = (RegExcmL3[1,0] + RegExcmL3[1,1])
REnL3 = (RegExcmL3[0,0] + RegExcmL3[0,1])
REfpL3 = RegExcmL3[1,0]
REfnL3 = RegExcmL3[0,1]
# accuracy: (tp + tn) / (p + n)
REaccuracyL3 = (REtpL3 + REtnL3) / (REpL3 + REnL3)
# precision tp / (tp + fp)
REprecisionL3 = REtpL3 / (REtpL3 + REfpL3)
RErecallL3 = REtpL3 / (REtpL3 + REfnL3)
# f1: 2 tp / (2 tp + fp + fn)
REf1L3 = 2*REtpL3 / (2*REtpL3 + REfpL3 + REfnL3)


def merge(Jthresh, J2thresh): 
      
    merged_list = [(Jthresh[i], J2thresh[i]) for i in range(0, len(Jthresh))] 
    return merged_list 

RegExLSA = merge(RegExthresh, LSAthresh)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
Combothresh = []
for value in RegExLSA:
    if value[0] or value[1] == 1:
        Combothresh.append(1)
    else:
        Combothresh.append(0)
			
RegExLSAI = merge(RegExthreshI, LSAthreshI)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
CombothreshI = []
for value in RegExLSAI:
    if value[0] or value[1] == 1:
        CombothreshI.append(1)
    else:
        CombothreshI.append(0)
			
RegExLSAL = merge(RegExthreshL, LSAthreshL)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
CombothreshL = []
for value in RegExLSAL:
    if value[0] or value[1] == 1:
        CombothreshL.append(1)
    else:
        CombothreshL.append(0)
        
Combocm = confusion_matrix(Jthresh, Combothresh)  #confusion matrix to plug into precision and recall tool
Ctp = Combocm[1,1]
Ctn = Combocm[0,0]
Cp = (Combocm[1,0] + Combocm[1,1])                    ##Combo vs Judge 1 Stringent
Cn = (Combocm[0,0] + Combocm[0,1])
Cfp = Combocm[1,0]
Cfn = Combocm[0,1]
# accuracy: (tp + tn) / (p + n)
Caccuracy = (Ctp + Ctn) / (Cp + Cn)
# precision tp / (tp + fp)
Cprecision = Ctp / (Ctp + Cfp)
    # recall: tp / (tp + fn)
Crecall = Ctp / (Ctp + Cfn)
# f1: 2 tp / (2 tp + fp + fn)
Cf1 = 2*Ctp / (2*Ctp + Cfp + Cfn)

Combocm2 = confusion_matrix(J2thresh, Combothresh)  #confusion matrix to plug into precision and recall tool
Ctp2 = Combocm2[1,1]
Ctn2 = Combocm2[0,0]
Cp2 = (Combocm2[1,0] + Combocm2[1,1])                    ##Combo vs Judge 2 stringent
Cn2 = (Combocm2[0,0] + Combocm2[0,1])
Cfp2 = Combocm2[1,0]
Cfn2 = Combocm2[0,1]
# accuracy: (tp + tn) / (p + n)
Caccuracy2 = (Ctp2 + Ctn2) / (Cp2 + Cn2)
# precision tp / (tp + fp)
Cprecision2 = Ctp2 / (Ctp2 + Cfp2)
    # recall: tp / (tp + fn)
Crecall2 = Ctp2 / (Ctp2 + Cfn2)
# f1: 2 tp / (2 tp + fp + fn)
Cf12 = 2*Ctp2 / (2*Ctp2 + Cfp2 + Cfn2)

Combocm3 = confusion_matrix(J3thresh, Combothresh)  #confusion matrix to plug into precision and recall tool
Ctp3 = Combocm3[1,1]
Ctn3 = Combocm3[0,0]
Cp3 = (Combocm3[1,0] + Combocm3[1,1])                    ##Combo vs Judge 3 stringent
Cn3 = (Combocm3[0,0] + Combocm3[0,1])
Cfp3 = Combocm3[1,0]
Cfn3 = Combocm3[0,1]
# accuracy: (tp + tn) / (p + n)
Caccuracy3 = (Ctp3 + Ctn3) / (Cp3 + Cn3)
# precision tp / (tp + fp)
Cprecision3 = Ctp3 / (Ctp3 + Cfp3)
    # recall: tp / (tp + fn)
Crecall3 = Ctp3 / (Ctp3 + Cfn3)
# f1: 2 tp / (2 tp + fp + fn)
Cf13 = 2*Ctp3 / (2*Ctp3 + Cfp3 + Cfn3)
    
CombocmI = confusion_matrix(JthreshI, CombothreshI)  #confusion matrix to plug into precision and recall tool
CtpI = CombocmI[1,1]
CtnI = CombocmI[0,0]
CpI = (CombocmI[1,0] + CombocmI[1,1])
CnI = (CombocmI[0,0] + CombocmI[0,1])
CfpI = CombocmI[1,0]
CfnI = CombocmI[0,1]
# accuracy: (tp + tn) / (p + n)
CaccuracyI = (CtpI + CtnI) / (CpI + CnI)
# precision tp / (tp + fp)
CprecisionI = CtpI / (CtpI + CfpI)
CrecallI = CtpI / (CtpI + CfnI)
# f1: 2 tp / (2 tp + fp + fn)
Cf1I = 2*CtpI / (2*CtpI + CfpI + CfnI)

CombocmI2 = confusion_matrix(J2threshI, CombothreshI)  #confusion matrix to plug into precision and recall tool
CtpI2 = CombocmI2[1,1]
CtnI2 = CombocmI2[0,0]
CpI2 = (CombocmI2[1,0] + CombocmI2[1,1])
CnI2 = (CombocmI2[0,0] + CombocmI2[0,1])
CfpI2 = CombocmI2[1,0]
CfnI2 = CombocmI2[0,1]
# accuracy: (tp + tn) / (p + n)
CaccuracyI2 = (CtpI2 + CtnI2) / (CpI2 + CnI2)
# precision tp / (tp + fp)
CprecisionI2 = CtpI2 / (CtpI2 + CfpI2)
CrecallI2 = CtpI2 / (CtpI2 + CfnI2)
# f1: 2 tp / (2 tp + fp + fn)
Cf1I2 = 2*CtpI2 / (2*CtpI2 + CfpI2 + CfnI2)

CombocmI3 = confusion_matrix(J3threshI, CombothreshI)  #confusion matrix to plug into precision and recall tool
CtpI3 = CombocmI3[1,1]
CtnI3 = CombocmI3[0,0]
CpI3 = (CombocmI3[1,0] + CombocmI3[1,1])
CnI3 = (CombocmI3[0,0] + CombocmI3[0,1])
CfpI3 = CombocmI3[1,0]
CfnI3 = CombocmI3[0,1]
# accuracy: (tp + tn) / (p + n)
CaccuracyI3 = (CtpI3 + CtnI3) / (CpI3 + CnI3)
# precision tp / (tp + fp)
CprecisionI3 = CtpI3 / (CtpI3 + CfpI3)
CrecallI3 = CtpI3 / (CtpI3 + CfnI3)
# f1: 2 tp / (2 tp + fp + fn)
Cf1I3 = 2*CtpI3 / (2*CtpI3 + CfpI3 + CfnI3)

CombocmL = confusion_matrix(JthreshL, CombothreshL)  #confusion matrix to plug into precision and recall tool
CtpL = CombocmL[1,1]
CtnL = CombocmL[0,0]
CpL = (CombocmL[1,0] + CombocmL[1,1])
CnL = (CombocmL[0,0] + CombocmL[0,1])
CfpL = CombocmL[1,0]
CfnL = CombocmL[0,1]
# accuracy: (tp + tn) / (p + n)
CaccuracyL = (CtpL + CtnL) / (CpL + CnL)
# precision tp / (tp + fp)
CprecisionL = CtpL / (CtpL + CfpL)
CrecallL = CtpL / (CtpL + CfnL)
# f1: 2 tp / (2 tp + fp + fn)
Cf1L = 2*CtpL / (2*CtpL + CfpL + CfnL)

CombocmL2 = confusion_matrix(J2threshL, CombothreshL)  #confusion matrix to plug into precision and recall tool
CtpL2 = CombocmL2[1,1]
CtnL2 = CombocmL2[0,0]
CpL2 = (CombocmL2[1,0] + CombocmL2[1,1])
CnL2 = (CombocmL2[0,0] + CombocmL2[0,1])
CfpL2 = CombocmL2[1,0]
CfnL2 = CombocmL2[0,1]
# accuracy: (tp + tn) / (p + n)
CaccuracyL2 = (CtpL2 + CtnL2) / (CpL2 + CnL2)
# precision tp / (tp + fp)
CprecisionL2 = CtpL2 / (CtpL2 + CfpL2)
CrecallL2 = CtpL2 / (CtpL2 + CfnL2)
# f1: 2 tp / (2 tp + fp + fn)
Cf1L2 = 2*CtpL2 / (2*CtpL2 + CfpL2 + CfnL2)

CombocmL3 = confusion_matrix(J3threshL, CombothreshL)  #confusion matrix to plug into precision and recall tool
CtpL3 = CombocmL3[1,1]
CtnL3 = CombocmL3[0,0]
CpL3 = (CombocmL3[1,0] + CombocmL3[1,1])
CnL3 = (CombocmL3[0,0] + CombocmL3[0,1])
CfpL3 = CombocmL3[1,0]
CfnL3 = CombocmL3[0,1]
# accuracy: (tp + tn) / (p + n)
CaccuracyL3 = (CtpL3 + CtnL3) / (CpL3 + CnL3)
# precision tp / (tp + fp)
CprecisionL3 = CtpL3 / (CtpL3 + CfpL3)
CrecallL3 = CtpL3 / (CtpL3 + CfnL3)
# f1: 2 tp / (2 tp + fp + fn)
Cf1L3 = 2*CtpL3 / (2*CtpL3 + CfpL3 + CfnL3)


In [13]:
table = EasyTable("My Table")
table.setCorners("/", "\\", "\\", "/")
table.setOuterStructure("|", "-")
table.setInnerStructure("|", "-", "+")
#Set the data in the table
table.setData([
    {"Judge 1 vs. LSA": "Stringent", "Kappa": round(cohen_kappa_score(Jthresh, LSAthresh), 3), "Precision%": (round(precision, 3)*100), "Recall%": (round(recall, 3)*100), "F1": round(f1, 3) },
    {"Judge 1 vs. LSA": "Intermediate", "Kappa": round(cohen_kappa_score(JthreshI, LSAthreshI), 3), "Precision%": (round(precisionI, 3)*100), "Recall%": (round(recallI, 3)*100), "F1": round(f1I, 3)},
    {"Judge 1 vs. LSA": "Lenient", "Kappa": round(cohen_kappa_score(JthreshL, LSAthreshL), 3), "Precision%": (round(precisionL, 3)*100), "Recall%": (round(recallL, 3)*100), "F1": round(f1L, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for LSA vs. Judge 1:")
table.displayTable()

#Set the data in the table
table.setData([
    {"Judge 2 vs. LSA": "Stringent", "Kappa": round(cohen_kappa_score(J2thresh, LSAthresh), 3), "Precision%": (round(precision2, 3)*100), "Recall%": (round(recall2, 3)*100), "F1": round(f12, 3) },
    {"Judge 2 vs. LSA": "Intermediate", "Kappa": round(cohen_kappa_score(J2threshI, LSAthreshI), 3), "Precision%": (round(precisionI2, 3)*100), "Recall%": (round(recallI2, 3)*100), "F1": round(f1I2, 3)},
    {"Judge 2 vs. LSA": "Lenient", "Kappa": round(cohen_kappa_score(J2threshL, LSAthreshL), 3), "Precision%": (round(precisionL2, 3)*100), "Recall%": (round(recallL2, 3)*100), "F1": round(f1L2, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for LSA vs. Judge 2:")
table.displayTable()

#Set the data in the table
table.setData([
    {"Judge 3 vs. LSA": "Stringent", "Kappa": round(cohen_kappa_score(J3thresh, LSAthresh), 3), "Precision%": (round(precision3, 3)*100), "Recall%": (round(recall3, 3)*100), "F1": round(f13, 3) },
    {"Judge 3 vs. LSA": "Intermediate", "Kappa": round(cohen_kappa_score(J3threshI, LSAthreshI), 3), "Precision%": (round(precisionI3, 3)*100), "Recall%": (round(recallI3, 3)*100), "F1": round(f1I3, 3)},
    {"Judge 3 vs. LSA": "Lenient", "Kappa": round(cohen_kappa_score(J3threshL, LSAthreshL), 3), "Precision%": (round(precisionL3, 3)*100), "Recall%": (round(recallL3, 3)*100), "F1": round(f1L3, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for LSA vs. Judge 3:")
table.displayTable()

Table for LSA vs. Judge 1:
/------------------------------------------------------\
|Judge 1 vs. LSA|Kappa|    Precision%    |Recall%| F1  |
|---------------+-----+------------------+-------+-----|
|   Stringent   |0.126|       19.8       | 29.4  |0.237|
|---------------+-----+------------------+-------+-----|
| Intermediate  |0.205|       57.4       | 61.4  |0.593|
|---------------+-----+------------------+-------+-----|
|    Lenient    |0.076|51.300000000000004| 93.0  |0.661|
\------------------------------------------------------/
Table for LSA vs. Judge 2:
/--------------------------------------------------------\
|Judge 1 vs. LSA|Kappa|Precision%|     Recall%     | F1  |
|---------------+-----+----------+-----------------+-----|
|               |0.116|   18.5   |      32.4       |0.235|
|---------------+-----+----------+-----------------+-----|
|               |0.129|   54.2   |      53.5       |0.538|
|---------------+-----+----------+-----------------+-----|
|               |0.0

641


In [15]:

#Set the data in the table
table.setData([
    {"J1 vs. J2": "Stringent", "Kappa": round(cohen_kappa_score(Jthresh, J2thresh), 3), "Precision%": (round(precisionJ12, 3)*100), "Recall%": (round(recallJ12, 3)*100), "F1": round(f1J12, 3) },
    {"J1 vs. J2": "Intermediate", "Kappa": round(cohen_kappa_score(JthreshI, J2threshI), 3), "Precision%": (round(precisionJI12, 3)*100), "Recall%": (round(recallJI12, 3)*100), "F1": round(f1JI12, 3)},
    {"J1 vs. J2": "Lenient", "Kappa": round(cohen_kappa_score(JthreshL, J2threshL), 3), "Precision%": (round(precisionJL12, 3)*100), "Recall%": (round(recallJL12, 3)*100), "F1": round(f1JL12, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for Judges 1 and 2:")
table.displayTable()

table = EasyTable("My Table")
table.setCorners("/", "\\", "\\", "/")
table.setOuterStructure("|", "-")
table.setInnerStructure("|", "-", "+")



#Set the data in the table
table.setData([
    {"J1 vs. J3": "Stringent", "Kappa": round(cohen_kappa_score(Jthresh, J3thresh), 3), "Precision%": (round(precisionJ13, 3)*100), "Recall%": (round(recallJ13, 3)*100), "F1": round(f1J13, 3) },
    {"J1 vs. J3": "Intermediate", "Kappa": round(cohen_kappa_score(JthreshI, J3threshI), 3), "Precision%": (round(precisionJI13, 3)*100), "Recall%": (round(recallJI13, 3)*100), "F1": round(f1JI13, 3)},
    {"J1 vs. J3": "Lenient", "Kappa": round(cohen_kappa_score(JthreshL, J3threshL), 3), "Precision%": (round(precisionJL13, 3)*100), "Recall%": (round(recallJL13, 3)*100), "F1": round(f1JL13, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for Judge 1 and 3:")
table.displayTable()

table = EasyTable("My Table")
table.setCorners("/", "\\", "\\", "/")
table.setOuterStructure("|", "-")
table.setInnerStructure("|", "-", "+")



#Set the data in the table
table.setData([
    {"J3 vs. J2": "Stringent", "Kappa": round(cohen_kappa_score(J3thresh, J2thresh), 3), "Precision%": (round(precisionJ23, 3)*100), "Recall%": (round(recallJ23, 3)*100), "F1": round(f1J23, 3) },
    {"J3 vs. J2": "Intermediate", "Kappa": round(cohen_kappa_score(J3threshI, J2threshI), 3), "Precision%": (round(precisionJI23, 3)*100), "Recall%": (round(recallJI23, 3)*100), "F1": round(f1JI23, 3)},
    {"J3 vs. J2": "Lenient", "Kappa": round(cohen_kappa_score(J3threshL, J2threshL), 3), "Precision%": (round(precisionJL23, 3)*100), "Recall%": (round(recallJL23, 3)*100), "F1": round(f1JL23, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for Judges:")
table.displayTable()

Table for Judges 1 and 2:
/-----------------------------------------------\
|J3 vs. J2|Kappa|   Precision%    |Recall%| F1  |
|---------+-----+-----------------+-------+-----|
|         |0.638|      76.2       | 64.7  | 0.7 |
|---------+-----+-----------------+-------+-----|
|         |0.635|78.10000000000001| 84.6  |0.812|
|---------+-----+-----------------+-------+-----|
|         |0.433|      97.7       | 92.7  |0.951|
\-----------------------------------------------/
Table for Judge 1 and 3:
/--------------------------------------------------\
| J1 vs. J3  |Kappa|   Precision%    |Recall%| F1  |
|------------+-----+-----------------+-------+-----|
| Stringent  |0.541|      76.2       | 53.1  |0.626|
|------------+-----+-----------------+-------+-----|
|Intermediate|0.553|      82.1       | 75.8  |0.788|
|------------+-----+-----------------+-------+-----|
|  Lenient   |0.322|77.60000000000001| 96.5  |0.86 |
\--------------------------------------------------/
Table for Judges:
/---

In [16]:
table = EasyTable("My Table")
table.setCorners("/", "\\", "\\", "/")
table.setOuterStructure("|", "-")
table.setInnerStructure("|", "-", "+")

#Set the data in the table
table.setData([
    {"Judge 1 vs. RegEx": "Stringent", "Kappa": round(cohen_kappa_score(Jthresh, RegExthresh), 3), "Precision%": (round(REprecision, 3)*100), "Recall%": (round(RErecall, 3)*100), "F1": round(REf1, 3) },
    {"Judge 1 vs. RegEx": "Intermediate", "Kappa": round(cohen_kappa_score(JthreshI, RegExthreshI), 3), "Precision%": (round(REprecisionI, 3)*100), "Recall%": (round(RErecallI, 3)*100), "F1": round(REf1I, 3)},
    {"Judge 1 vs. RegEx": "Lenient", "Kappa": round(cohen_kappa_score(JthreshL, RegExthreshL), 3), "Precision%": (round(REprecisionL, 3)*100), "Recall%": (round(RErecallL, 3)*100), "F1": round(REf1L, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for RegEx vs. Judge 1:")
table.displayTable()

#Set the data in the table
table.setData([
    {"Judge 2 vs. RegEx": "Stringent", "Kappa": round(cohen_kappa_score(J2thresh, RegExthresh), 3), "Precision%": (round(REprecision2, 3)*100), "Recall%": (round(RErecall2, 3)*100), "F1": round(REf12, 3) },
    {"Judge 2 vs. RegEx": "Intermediate", "Kappa": round(cohen_kappa_score(J2threshI, RegExthreshI), 3), "Precision%": (round(REprecisionI2, 3)*100), "Recall%": (round(RErecallI2, 3)*100), "F1": round(REf1I2, 3)},
    {"Judge 2 vs. RegEx": "Lenient", "Kappa": round(cohen_kappa_score(J2threshL, RegExthreshL), 3), "Precision%": (round(REprecisionL2, 3)*100), "Recall%": (round(RErecallL2, 3)*100), "F1": round(REf1L2, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for RegEx vs. Judge 2:")
table.displayTable()

#Set the data in the table
table.setData([
    {"Judge 3 vs. RegEx": "Stringent", "Kappa": round(cohen_kappa_score(J3thresh, RegExthresh), 3), "Precision%": (round(REprecision3, 3)*100), "Recall%": (round(RErecall3, 3)*100), "F1": round(REf13, 3) },
    {"Judge 3 vs. RegEx": "Intermediate", "Kappa": round(cohen_kappa_score(J3threshI, RegExthreshI), 3), "Precision%": (round(REprecisionI3, 3)*100), "Recall%": (round(RErecallI3, 3)*100), "F1": round(REf1I3, 3)},
    {"Judge 3 vs. RegEx": "Lenient", "Kappa": round(cohen_kappa_score(J3threshL, RegExthreshL), 3), "Precision%": (round(REprecisionL3, 3)*100), "Recall%": (round(RErecallL3, 3)*100), "F1": round(REf1L3, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for RegEx vs. Judge 3:")
table.displayTable()

Table for RegEx vs. Judge 1:
/------------------------------------------------\
|Judge 1 vs. RegEx|Kappa|Precision%|Recall%| F1  |
|-----------------+-----+----------+-------+-----|
|    Stringent    |0.254|   28.7   | 43.9  |0.347|
|-----------------+-----+----------+-------+-----|
|  Intermediate   |0.24 |   34.9   | 76.9  |0.48 |
|-----------------+-----+----------+-------+-----|
|     Lenient     |0.12 |   50.8   | 95.7  |0.664|
\------------------------------------------------/
Table for RegEx vs. Judge 2:
/--------------------------------------------------------\
|Judge 1 vs. RegEx|Kappa|    Precision%    |Recall%| F1  |
|-----------------+-----+------------------+-------+-----|
|                 |0.259|27.700000000000003| 50.0  |0.357|
|-----------------+-----+------------------+-------+-----|
|                 |0.288|       37.8       | 76.9  |0.507|
|-----------------+-----+------------------+-------+-----|
|                 |0.068|       49.2       | 97.7  |0.654|
\----------

In [17]:
table = EasyTable("My Table")
table.setCorners("/", "\\", "\\", "/")
table.setOuterStructure("|", "-")
table.setInnerStructure("|", "-", "+")

#Set the data in the table
table.setData([
    {"Judge 1 vs. Combo": "Stringent", "Kappa": round(cohen_kappa_score(Jthresh, Combothresh), 3), "Precision%": (round(Cprecision, 3)*100), "Recall%": (round(Crecall, 3)*100), "F1": round(Cf1, 3) },
    {"Judge 1 vs. Combo": "Intermediate", "Kappa": round(cohen_kappa_score(JthreshI, CombothreshI), 3), "Precision%": (round(CprecisionI, 3)*100), "Recall%": (round(CrecallI, 3)*100), "F1": round(Cf1I, 3)},
    {"Judge 1 vs. Combo": "Lenient", "Kappa": round(cohen_kappa_score(JthreshL, CombothreshL), 3), "Precision%": (round(CprecisionL, 3)*100), "Recall%": (round(CrecallL, 3)*100), "F1": round(Cf1L, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for Combo vs. Judge 1:")
table.displayTable()

#Set the data in the table
table.setData([
    {"Judge 2 vs. Combo": "Stringent", "Kappa": round(cohen_kappa_score(J2thresh, Combothresh), 3), "Precision%": (round(Cprecision2, 3)*100), "Recall%": (round(Crecall2, 3)*100), "F1": round(Cf12, 3) },
    {"Judge 2 vs. Combo": "Intermediate", "Kappa": round(cohen_kappa_score(J2threshI, CombothreshI), 3), "Precision%": (round(CprecisionI2, 3)*100), "Recall%": (round(CrecallI2, 3)*100), "F1": round(Cf1I2, 3)},
    {"Judge 2 vs. Combo": "Lenient", "Kappa": round(cohen_kappa_score(J2threshL, CombothreshL), 3), "Precision%": (round(CprecisionL2, 3)*100), "Recall%": (round(CrecallL2, 3)*100), "F1": round(Cf1L2, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for Combo vs. Judge 2:")
table.displayTable()

#Set the data in the table
table.setData([
    {"Judge 3 vs. Combo": "Stringent", "Kappa": round(cohen_kappa_score(J3thresh, Combothresh), 3), "Precision%": (round(Cprecision3, 3)*100), "Recall%": (round(Crecall3, 3)*100), "F1": round(Cf13, 3) },
    {"Judge 3 vs. Combo": "Intermediate", "Kappa": round(cohen_kappa_score(J3threshI, CombothreshI), 3), "Precision%": (round(CprecisionI3, 3)*100), "Recall%": (round(CrecallI3, 3)*100), "F1": round(Cf1I3, 3)},
    {"Judge 3 vs. Combo": "Lenient", "Kappa": round(cohen_kappa_score(J3threshL, CombothreshL), 3), "Precision%": (round(CprecisionL3, 3)*100), "Recall%": (round(CrecallL3, 3)*100), "F1": round(Cf1L3, 3)}
  ])

#Display the table for judges/computer performance metrics
print("Table for Combo vs. Judge 3:")
table.displayTable()

Table for Combo vs. Judge 1:
/-------------------------------------------------------\
|Judge 1 vs. Combo|Kappa|   Precision%    |Recall%| F1  |
|-----------------+-----+-----------------+-------+-----|
|    Stringent    |0.224|      38.6       | 33.1  |0.356|
|-----------------+-----+-----------------+-------+-----|
|  Intermediate   |0.244|      68.2       | 61.4  |0.646|
|-----------------+-----+-----------------+-------+-----|
|     Lenient     |0.161|70.39999999999999| 93.5  |0.803|
\-------------------------------------------------------/
Table for Combo vs. Judge 2:
/-----------------------------------------------------------\
|Judge 1 vs. Combo|Kappa|Precision%|     Recall%      | F1  |
|-----------------+-----+----------+------------------+-----|
|                 |0.198|   34.5   |34.699999999999996|0.346|
|-----------------+-----+----------+------------------+-----|
|                 |0.205|   67.2   |55.800000000000004|0.61 |
|-----------------+-----+----------+------------

In [21]:
listStringentJudge1 = []
listStringentJudge1P = []
listStringentJudge1R = []
ClistStringentJudge1 = []
ClistStringentJudge1P = []
ClistStringentJudge1R = []
i = 0
for value in range(1000):
    i += .001
    Jthresh = []
    for value in NAVY.J1:                        ##the next 8 for loops are about coding match values for human judges and computer models
        if value == Jthreshv: 
            Jthresh.append(1)  
        else: 
            Jthresh.append(0)  
    LSAthresh = [] 
    for value in df['LSA']: 
        if value >= i: 
            LSAthresh.append(1)
        else: 
            LSAthresh.append(0)
    RegExLSA = merge(RegExthresh, LSAthresh)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
    Combothresh = []
    for value in RegExLSA:
        if value[0] or value[1] == 1:
            Combothresh.append(1)
        else:
            Combothresh.append(0)

 
    LSAcm = confusion_matrix(Jthresh, LSAthresh)  #confusion matrix to plug into precision and recall tool
    tp = LSAcm[1,1]
    tn = LSAcm[0,0]
    p = (LSAcm[1,0] + LSAcm[1,1])                    ##LSA vs Judge 1
    n = (LSAcm[0,0] + LSAcm[0,1])
    fp = LSAcm[1,0]
    fn = LSAcm[0,1]
# accuracy: (tp + tn) / (p + n)
    accuracy = (tp + tn) / (p + n)
# precision tp / (tp + fp)
    precision = tp / (tp + fp)
    # recall: tp / (tp + fn)
    recall = tp / (tp + fn)
# f1: 2 tp / (2 tp + fp + fn)
    f1 = 2*tp / (2*tp + fp + fn)
    listStringentJudge1.append((f1, i))
    listStringentJudge1P.append((precision, i))
    listStringentJudge1R.append((recall, i))
    
    Combocm = confusion_matrix(Jthresh, Combothresh)  #confusion matrix to plug into precision and recall tool
    Ctp = Combocm[1,1]
    Ctn = Combocm[0,0]
    Cp = (Combocm[1,0] + Combocm[1,1])                    ##Combo vs Judge 1
    Cn = (Combocm[0,0] + Combocm[0,1])
    Cfp = Combocm[1,0]
    Cfn = Combocm[0,1]
# accuracy: (tp + tn) / (p + n)
    Caccuracy = (Ctp + Ctn) / (Cp + Cn)
# precision tp / (tp + fp)
    Cprecision = Ctp / (Ctp + Cfp)
    # recall: tp / (tp + fn)
    Crecall = Ctp / (Ctp + Cfn)
# f1: 2 tp / (2 tp + fp + fn)
    Cf1 = 2*Ctp / (2*Ctp + Cfp + Cfn)
    ClistStringentJudge1.append((Cf1, i))
    ClistStringentJudge1P.append((Cprecision, i))
    ClistStringentJudge1R.append((Crecall, i))
    
listLenientJudge2P = []
listLenientJudge2R = []
listLenientJudge2 = []
ClistLenientJudge2P = []
ClistLenientJudge2R = []
ClistLenientJudge2 = []
i = 0
for value in range(1000):
    i += .001
    J2threshL = [] 
    for value in NAVY.J2: 
        if value >= J2threshLv: 
            J2threshL.append(1)  
        else: 
            J2threshL.append(0)     
    LSAthreshL = [] 
    for value in df['LSA']: 
        if value >= i: 
            LSAthreshL.append(1)
        else: 
            LSAthreshL.append(0)
    RegExLSAL = merge(RegExthreshL, LSAthreshL)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
    CombothreshL = []
    for value in RegExLSAL:
        if value[0] or value[1] == 1:
            CombothreshL.append(1)
        else:
            CombothreshL.append(0)
        

    LSAcmL2 = confusion_matrix(J2threshL, LSAthreshL)  #confusion matrix to plug into precision and recall tool
    tpL2 = LSAcmL2[1,1]
    tnL2 = LSAcmL2[0,0]
    pL2 = (LSAcmL2[1,0] + LSAcmL2[1,1])
    nL2 = (LSAcmL2[0,0] + LSAcmL2[0,1])
    fpL2 = LSAcmL2[1,0]
    fnL2 = LSAcmL2[0,1]
# accuracy: (tp + tn) / (p + n)
    accuracyL2 = (tpL2 + tnL2) / (pL2 + nL2)
# precision tp / (tp + fp)
    precisionL2 = tpL2 / (tpL2 + fpL2)
    recallL2 = tpL2 / (tpL2 + fnL2)
# f1: 2 tp / (2 tp + fp + fn)
    f1L2 = 2*tpL2 / (2*tpL2 + fpL2 + fnL2)
    listLenientJudge2.append((f1L2, i))
    listLenientJudge2P.append((precisionL2, i))
    listLenientJudge2R.append((recallL2, i))
    
    CombocmL2 = confusion_matrix(J2threshL, CombothreshL)  #confusion matrix to plug into precision and recall tool
    CtpL2 = CombocmL2[1,1]
    CtnL2 = CombocmL2[0,0]
    CpL2 = (CombocmL2[1,0] + CombocmL2[1,1])
    CnL2 = (CombocmL2[0,0] + CombocmL2[0,1])
    CfpL2 = CombocmL2[1,0]
    CfnL2 = CombocmL2[0,1]
# accuracy: (tp + tn) / (p + n)
    CaccuracyL2 = (CtpL2 + CtnL2) / (CpL2 + CnL2)
# precision tp / (tp + fp)
    CprecisionL2 = CtpL2 / (CtpL2 + CfpL2)
    CrecallL2 = CtpL2 / (CtpL2 + CfnL2)
# f1: 2 tp / (2 tp + fp + fn)
    Cf1L2 = 2*CtpL2 / (2*CtpL2 + CfpL2 + CfnL2)
    ClistLenientJudge2.append((Cf1L2, i))
    ClistLenientJudge2P.append((CprecisionL2, i))
    ClistLenientJudge2R.append((CrecallL2, i))
    
listLenientJudge1P = []
listLenientJudge1R = []
listLenientJudge1 = []
ClistLenientJudge1P = []
ClistLenientJudge1R = []
ClistLenientJudge1 = []
i = 0
for value in range(1000):
    i += .001
    JthreshL = []                        ##the next 8 for loops are about coding match values for human judges and computer models in lenient thresholds.
    for value in NAVY.J1: 
        if value >= JthreshLv: 
            JthreshL.append(1)  
        else: 
            JthreshL.append(0)   
    LSAthreshL = [] 
    for value in df['LSA']: 
        if value >= i: 
            LSAthreshL.append(1)
        else: 
            LSAthreshL.append(0)
    RegExLSAL = merge(RegExthreshL, LSAthreshL)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
    CombothreshL = []
    for value in RegExLSAL:
        if value[0] or value[1] == 1:
            CombothreshL.append(1)
        else:
            CombothreshL.append(0)

    LSAcmL = confusion_matrix(JthreshL, LSAthreshL)  #confusion matrix to plug into precision and recall tool
    tpL = LSAcmL[1,1]
    tnL = LSAcmL[0,0]
    pL = (LSAcmL[1,0] + LSAcmL[1,1])
    nL = (LSAcmL[0,0] + LSAcmL[0,1])
    fpL = LSAcmL[1,0]
    fnL = LSAcmL[0,1]
# accuracy: (tp + tn) / (p + n)
    accuracyL = (tpL + tnL) / (pL + nL)
# precision tp / (tp + fp)
    precisionL = tpL / (tpL + fpL)
    recallL = tpL / (tpL + fnL)
# f1: 2 tp / (2 tp + fp + fn)
    f1L = 2*tpL / (2*tpL + fpL + fnL)
    listLenientJudge1.append((f1L, i))
    listLenientJudge1P.append((precisionL, i))
    listLenientJudge1R.append((recallL, i))
    CombocmL = confusion_matrix(JthreshL, CombothreshL)  #confusion matrix to plug into precision and recall tool
    CtpL = CombocmL[1,1]
    CtnL = CombocmL[0,0]
    CpL = (CombocmL[1,0] + CombocmL[1,1])
    CnL = (CombocmL[0,0] + CombocmL[0,1])
    CfpL = CombocmL[1,0]
    CfnL = CombocmL[0,1]
# accuracy: (tp + tn) / (p + n)
    CaccuracyL = (CtpL + CtnL) / (CpL + CnL)
# precision tp / (tp + fp)
    CprecisionL = CtpL / (CtpL + CfpL)
    CrecallL = CtpL / (CtpL + CfnL)
# f1: 2 tp / (2 tp + fp + fn)
    Cf1L = 2*CtpL / (2*CtpL + CfpL + CfnL)
    ClistLenientJudge1.append((Cf1L, i))
    ClistLenientJudge1P.append((CprecisionL, i))
    ClistLenientJudge1R.append((CrecallL, i))
    

listIntermediateJudge2 = []
listIntermediateJudge2P = []
listIntermediateJudge2R = []
ClistIntermediateJudge2 = []
ClistIntermediateJudge2P = []
ClistIntermediateJudge2R = []
i = 0
for value in range(1000):
    i += .001     
    J2threshI = [] 
    for value in NAVY.J2: 
        if value >= J2threshIv: 
            J2threshI.append(1)  
        else: 
            J2threshI.append(0)
    LSAthreshI = [] 
    for value in df['LSA']: 
        if value >= i: 
            LSAthreshI.append(1)
        else: 
            LSAthreshI.append(0)
    RegExLSAI = merge(RegExthreshI, LSAthreshI)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
    CombothreshI = []
    for value in RegExLSAI:
        if value[0] or value[1] == 1:
            CombothreshI.append(1)
        else:
            CombothreshI.append(0)

    LSAcmI2 = confusion_matrix(J2threshI, LSAthreshI)  #confusion matrix to plug into precision and recall tool
    tpI2 = LSAcmI2[1,1]
    tnI2 = LSAcmI2[0,0]
    pI2 = (LSAcmI2[1,0] + LSAcmI2[1,1])
    nI2 = (LSAcmI2[0,0] + LSAcmI2[0,1])
    fpI2 = LSAcmI2[1,0]
    fnI2 = LSAcmI2[0,1]
# accuracy: (tp + tn) / (p + n)
    accuracyI2 = (tpI2 + tnI2) / (pI2 + nI2)
# precision tp / (tp + fp)
    precisionI2 = tpI2 / (tpI2 + fpI2)
# recall: tp / (tp + fn)
    recallI2 = tpI2 / (tpI2 + fnI2)
# f1: 2 tp / (2 tp + fp + fn)
    f1I2 = 2*tpI2 / (2*tpI2 + fpI2 + fnI2)
    listIntermediateJudge2.append((f1I2, i))
    listIntermediateJudge2P.append((precisionI2, i))
    listIntermediateJudge2R.append((recallI2, i))
    
    CombocmI2 = confusion_matrix(J2threshI, CombothreshI)  #confusion matrix to plug into precision and recall tool
    CtpI2 = CombocmI2[1,1]
    CtnI2 = CombocmI2[0,0]
    CpI2 = (CombocmI2[1,0] + CombocmI2[1,1])
    CnI2 = (CombocmI2[0,0] + CombocmI2[0,1])
    CfpI2 = CombocmI2[1,0]
    CfnI2 = CombocmI2[0,1]
# accuracy: (tp + tn) / (p + n)
    CaccuracyI2 = (CtpI2 + CtnI2) / (CpI2 + CnI2)
# precision tp / (tp + fp)
    CprecisionI2 = CtpI2 / (CtpI2 + CfpI2)
    CrecallI2 = CtpI2 / (CtpI2 + CfnI2)
# f1: 2 tp / (2 tp + fp + fn)
    Cf1I2 = 2*CtpI2 / (2*CtpI2 + CfpI2 + CfnI2)
    ClistIntermediateJudge2.append((Cf1I2, i))
    ClistIntermediateJudge2P.append((CprecisionI2, i))
    ClistIntermediateJudge2R.append((CrecallI2, i))


listIntermediateJudge1 = []
listIntermediateJudge1P = []
listIntermediateJudge1R = []
ClistIntermediateJudge1 = []
ClistIntermediateJudge1P = []
ClistIntermediateJudge1R = []
i = 0
for value in range(1000):
    i += .001     
    JthreshI = []                        ##the next 8 for loops are about coding match values for human judges and computer models in intermediate thresholds.
    for value in NAVY.J1: 
        if value >= JthreshIv: 
            JthreshI.append(1)  
        else: 
            JthreshI.append(0) 
    LSAthreshI = [] 
    for value in df['LSA']: 
        if value >= i: 
            LSAthreshI.append(1)
        else: 
            LSAthreshI.append(0) 
    RegExLSAI = merge(RegExthreshI, LSAthreshI)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
    CombothreshI = []
    for value in RegExLSAI:
        if value[0] or value[1] == 1:
            CombothreshI.append(1)
        else:
            CombothreshI.append(0)

    LSAcmI = confusion_matrix(JthreshI, LSAthreshI)  #confusion matrix to plug into precision and recall tool
    tpI = LSAcmI[1,1]
    tnI = LSAcmI[0,0]
    pI = (LSAcmI[1,0] + LSAcmI[1,1])
    nI = (LSAcmI[0,0] + LSAcmI[0,1])
    fpI = LSAcmI[1,0]
    fnI = LSAcmI[0,1]
# accuracy: (tp + tn) / (p + n)
    accuracyI = (tpI + tnI) / (pI + nI)
# precision tp / (tp + fp)
    precisionI = tpI / (tpI + fpI)
# recall: tp / (tp + fn)
    recallI = tpI / (tpI + fnI)
    # f1: 2 tp / (2 tp + fp + fn)
    f1I = 2*tpI / (2*tpI + fpI + fnI)
    listIntermediateJudge1.append((f1I, i))
    listIntermediateJudge1R.append((recallI, i))
    listIntermediateJudge1P.append((precisionI, i))
    
    CombocmI = confusion_matrix(JthreshI, CombothreshI)  #confusion matrix to plug into precision and recall tool
    CtpI = CombocmI[1,1]
    CtnI = CombocmI[0,0]
    CpI = (CombocmI[1,0] + CombocmI[1,1])
    CnI = (CombocmI[0,0] + CombocmI[0,1])
    CfpI = CombocmI[1,0]
    CfnI = CombocmI[0,1]
# accuracy: (tp + tn) / (p + n)
    CaccuracyI = (CtpI + CtnI) / (CpI + CnI)
# precision tp / (tp + fp)
    CprecisionI = CtpI / (CtpI + CfpI)
    CrecallI = CtpI / (CtpI + CfnI)
# f1: 2 tp / (2 tp + fp + fn)
    Cf1I = 2*CtpI / (2*CtpI + CfpI + CfnI)
    ClistIntermediateJudge1.append((Cf1I, i))
    ClistIntermediateJudge1P.append((CprecisionI, i))
    ClistIntermediateJudge1R.append((CrecallI, i))
    
    
listStringentJudge2R = []
listStringentJudge2P = []
listStringentJudge2 = []
ClistStringentJudge2R = []
ClistStringentJudge2P = []
ClistStringentJudge2 = []
i = 0
for value in range(1000):
    i += .001
    J2thresh = [] 
    for value in NAVY.J2: 
        if value == J2threshv: 
            J2thresh.append(1)  
        else: 
            J2thresh.append(0)    
    LSAthresh = [] 
    for value in df['LSA']: 
        if value >= i: 
            LSAthresh.append(1)
        else: 
            LSAthresh.append(0)
    RegExLSA = merge(RegExthresh, LSAthresh)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
    Combothresh = []
    for value in RegExLSA:
        if value[0] or value[1] == 1:
            Combothresh.append(1)
        else:
            Combothresh.append(0)

    LSAcm2 = confusion_matrix(J2thresh, LSAthresh)  #confusion matrix to plug into precision and recall tool
    tp2 = LSAcm2[1,1]
    tn2 = LSAcm2[0,0]
    p2 = (LSAcm2[1,0] + LSAcm2[1,1])                    ##LSA vs Judge 2
    n2 = (LSAcm2[0,0] + LSAcm2[0,1])
    fp2 = LSAcm2[1,0]
    fn2 = LSAcm2[0,1]
# accuracy: (tp + tn) / (p + n)
    accuracy2 = (tp2 + tn2) / (p2 + n2)
# precision tp / (tp + fp)
    precision2 = tp2 / (tp2 + fp2)
# recall: tp / (tp + fn)
    recall2 = tp2 / (tp2 + fn2)
# f1: 2 tp / (2 tp + fp + fn)
    f12 = 2*tp2 / (2*tp2 + fp2 + fn2)
    listStringentJudge2.append((f12, i))
    listStringentJudge2R.append((recall2, i))
    listStringentJudge2P.append((precision2, i))

    Combocm2 = confusion_matrix(J2thresh, Combothresh)  #confusion matrix to plug into precision and recall tool
    Ctp2 = Combocm2[1,1]
    Ctn2 = Combocm2[0,0]
    Cp2 = (Combocm2[1,0] + Combocm2[1,1])                    ##Combo vs Judge 1
    Cn2 = (Combocm2[0,0] + Combocm2[0,1])
    Cfp2 = Combocm2[1,0]
    Cfn2 = Combocm2[0,1]
# accuracy: (tp + tn) / (p + n)
    Caccuracy2 = (Ctp2 + Ctn2) / (Cp2 + Cn2)
# precision tp / (tp + fp)
    Cprecision2 = Ctp2 / (Ctp2 + Cfp2)
    # recall: tp / (tp + fn)
    Crecall2 = Ctp2 / (Ctp2 + Cfn2)
# f1: 2 tp / (2 tp + fp + fn)
    Cf12 = 2*Ctp2 / (2*Ctp2 + Cfp2 + Cfn2)
    ClistStringentJudge2.append((Cf12, i))
    ClistStringentJudge2P.append((Cprecision2, i))
    ClistStringentJudge2R.append((Crecall2, i))

listStringentJudge3R = []
listStringentJudge3P = []
listStringentJudge3 = []
ClistStringentJudge3R = []
ClistStringentJudge3P = []
ClistStringentJudge3 = []
i = 0
for value in range(1000):
    i += .001
    J2thresh = [] 
    for value in NAVY.J3: 
        if value == J3threshv: 
            J3thresh.append(1)  
        else: 
            J3thresh.append(0)    
    LSAthresh = [] 
    for value in df['LSA']: 
        if value >= i: 
            LSAthresh.append(1)
        else: 
            LSAthresh.append(0)
    RegExLSA = merge(RegExthresh, LSAthresh)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
    Combothresh = []
    for value in RegExLSA:
        if value[0] or value[1] == 1:
            Combothresh.append(1)
        else:
            Combothresh.append(0)

    LSAcm3 = confusion_matrix(J3thresh, LSAthresh)  #confusion matrix to plug into precision and recall tool
    tp3 = LSAcm3[1,1]
    tn3 = LSAcm3[0,0]
    p3 = (LSAcm3[1,0] + LSAcm3[1,1])                    ##LSA vs Judge 3
    n3 = (LSAcm3[0,0] + LSAcm3[0,1])
    fp3 = LSAcm3[1,0]
    fn3 = LSAcm3[0,1]
# accuracy: (tp + tn) / (p + n)
    accuracy3 = (tp3 + tn3) / (p3 + n3)
# precision tp / (tp + fp)
    precision3 = tp3 / (tp3 + fp3)
# recall: tp / (tp + fn)
    recall3 = tp3 / (tp3 + fn3)
# f1: 2 tp / (2 tp + fp + fn)
    f13 = 2*tp3 / (2*tp3 + fp3 + fn3)
    listStringentJudge3.append((f13, i))
    listStringentJudge3R.append((recall3, i))
    listStringentJudge3P.append((precision3, i))

    Combocm3 = confusion_matrix(J3thresh, Combothresh)  #confusion matrix to plug into precision and recall tool
    Ctp3 = Combocm3[1,1]
    Ctn3 = Combocm3[0,0]
    Cp3 = (Combocm3[1,0] + Combocm3[1,1])                    ##Combo vs Judge 1
    Cn3 = (Combocm3[0,0] + Combocm3[0,1])
    Cfp3 = Combocm3[1,0]
    Cfn3 = Combocm3[0,1]
# accuracy: (tp + tn) / (p + n)
    Caccuracy3 = (Ctp3 + Ctn3) / (Cp3 + Cn3)
# precision tp / (tp + fp)
    Cprecision3 = Ctp3 / (Ctp3 + Cfp3)
    # recall: tp / (tp + fn)
    Crecall3 = Ctp3 / (Ctp3 + Cfn3)
# f1: 2 tp / (2 tp + fp + fn)
    Cf13 = 2*Ctp3 / (2*Ctp3 + Cfp3 + Cfn3)
    ClistStringentJudge3.append((Cf13, i))
    ClistStringentJudge3P.append((Cprecision3, i))
    ClistStringentJudge3R.append((Crecall3, i))


listLenientJudge3P = []
listLenientJudge3R = []
listLenientJudge3 = []
ClistLenientJudge3P = []
ClistLenientJudge3R = []
ClistLenientJudge3 = []
i = 0
for value in range(1000):
    i += .001
    J3threshL = [] 
    for value in NAVY.J3: 
        if value >= J3threshLv: 
            J3threshL.append(1)  
        else: 
            J3threshL.append(0)     
    LSAthreshL = [] 
    for value in df['LSA']: 
        if value >= i: 
            LSAthreshL.append(1)
        else: 
            LSAthreshL.append(0)
    RegExLSAL = merge(RegExthreshL, LSAthreshL)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
    CombothreshL = []
    for value in RegExLSAL:
        if value[0] or value[1] == 1:
            CombothreshL.append(1)
        else:
            CombothreshL.append(0)
        

    LSAcmL3 = confusion_matrix(J3threshL, LSAthreshL)  #confusion matrix to plug into precision and recall tool
    tpL3 = LSAcmL3[1,1]
    tnL3 = LSAcmL3[0,0]
    pL3 = (LSAcmL3[1,0] + LSAcmL3[1,1])
    nL3 = (LSAcmL3[0,0] + LSAcmL3[0,1])
    fpL3 = LSAcmL3[1,0]
    fnL3 = LSAcmL3[0,1]
# accuracy: (tp + tn) / (p + n)
    accuracyL3 = (tpL3 + tnL3) / (pL3 + nL3)
# precision tp / (tp + fp)
    precisionL3 = tpL3 / (tpL3 + fpL3)
    recallL3 = tpL3 / (tpL3 + fnL3)
# f1: 2 tp / (2 tp + fp + fn)
    f1L3 = 2*tpL3 / (2*tpL3 + fpL3 + fnL3)
    listLenientJudge3.append((f1L3, i))
    listLenientJudge3P.append((precisionL3, i))
    listLenientJudge3R.append((recallL3, i))
    
    CombocmL3 = confusion_matrix(J3threshL, CombothreshL)  #confusion matrix to plug into precision and recall tool
    CtpL3 = CombocmL3[1,1]
    CtnL3 = CombocmL3[0,0]
    CpL3 = (CombocmL3[1,0] + CombocmL3[1,1])
    CnL3 = (CombocmL3[0,0] + CombocmL3[0,1])
    CfpL3 = CombocmL3[1,0]
    CfnL3 = CombocmL3[0,1]
# accuracy: (tp + tn) / (p + n)
    CaccuracyL3 = (CtpL3 + CtnL3) / (CpL3 + CnL3)
# precision tp / (tp + fp)
    CprecisionL3 = CtpL3 / (CtpL3 + CfpL3)
    CrecallL3 = CtpL3 / (CtpL3 + CfnL3)
# f1: 2 tp / (2 tp + fp + fn)
    Cf1L3 = 2*CtpL3 / (2*CtpL3 + CfpL3 + CfnL3)
    ClistLenientJudge3.append((Cf1L3, i))
    ClistLenientJudge3P.append((CprecisionL3, i))
    ClistLenientJudge3R.append((CrecallL3, i))
	
	
	
listIntermediateJudge3 = []
listIntermediateJudge3P = []
listIntermediateJudge3R = []
ClistIntermediateJudge3 = []
ClistIntermediateJudge3P = []
ClistIntermediateJudge3R = []
i = 0
for value in range(1000):
    i += .001     
    J3threshI = [] 
    for value in NAVY.J3: 
        if value >= J3threshIv: 
            J3threshI.append(1)  
        else: 
            J3threshI.append(0)
    LSAthreshI = [] 
    for value in df['LSA']: 
        if value >= i: 
            LSAthreshI.append(1)
        else: 
            LSAthreshI.append(0)
    RegExLSAI = merge(RegExthreshI, LSAthreshI)           ##Very important. Used for Judges either/or as well as LSA/RegEx combination thresholds.
    CombothreshI = []
    for value in RegExLSAI:
        if value[0] or value[1] == 1:
            CombothreshI.append(1)
        else:
            CombothreshI.append(0)

    LSAcmI3 = confusion_matrix(J3threshI, LSAthreshI)  #confusion matrix to plug into precision and recall tool
    tpI3 = LSAcmI3[1,1]
    tnI3 = LSAcmI3[0,0]
    pI3 = (LSAcmI3[1,0] + LSAcmI3[1,1])
    nI3 = (LSAcmI3[0,0] + LSAcmI3[0,1])
    fpI3 = LSAcmI3[1,0]
    fnI3 = LSAcmI3[0,1]
# accuracy: (tp + tn) / (p + n)
    accuracyI3 = (tpI3 + tnI3) / (pI3 + nI3)
# precision tp / (tp + fp)
    precisionI3 = tpI3 / (tpI3 + fpI3)
# recall: tp / (tp + fn)
    recallI3 = tpI3 / (tpI3 + fnI3)
# f1: 2 tp / (2 tp + fp + fn)
    f1I3 = 2*tpI3 / (2*tpI3 + fpI3 + fnI3)
    listIntermediateJudge3.append((f1I3, i))
    listIntermediateJudge3P.append((precisionI3, i))
    listIntermediateJudge3R.append((recallI3, i))
    
    CombocmI3 = confusion_matrix(J3threshI, CombothreshI)  #confusion matrix to plug into precision and recall tool
    CtpI3 = CombocmI3[1,1]
    CtnI3 = CombocmI3[0,0]
    CpI3 = (CombocmI3[1,0] + CombocmI3[1,1])
    CnI3 = (CombocmI3[0,0] + CombocmI3[0,1])
    CfpI3 = CombocmI3[1,0]
    CfnI3 = CombocmI3[0,1]
# accuracy: (tp + tn) / (p + n)
    CaccuracyI3 = (CtpI3 + CtnI3) / (CpI3 + CnI3)
# precision tp / (tp + fp)
    CprecisionI3 = CtpI3 / (CtpI3 + CfpI3)
    CrecallI3 = CtpI3 / (CtpI3 + CfnI3)
# f1: 2 tp / (2 tp + fp + fn)
    Cf1I3 = 2*CtpI3 / (2*CtpI3 + CfpI3 + CfnI3)
    ClistIntermediateJudge3.append((Cf1I3, i))
    ClistIntermediateJudge3P.append((CprecisionI3, i))
    ClistIntermediateJudge3R.append((CrecallI3, i))

<class 'ValueError'>: Found input variables with inconsistent numbers of samples: [1282, 641]

In [ ]:
print('------------------------')
print('LSA(S) vs. Judge 1- F1:', max([(x[0], x[1]) for x in listStringentJudge1]))
print('LSA(S) vs. Judge 2- F1:', max([(x[0], x[1]) for x in listStringentJudge2]))


print('------------------------')
print('LSA(I) vs. Judge 1- F1:', max([(x[0], x[1]) for x in listIntermediateJudge1]))
print('LSA(I) vs. Judge 2- F1:', max([(x[0], x[1]) for x in listIntermediateJudge2]))


print('------------------------')
print('LSA(L) vs. Judge 1- F1:', max([(x[0], x[1]) for x in listLenientJudge1]))
print('LSA(L) vs. Judge 2- F1:', max([(x[0], x[1]) for x in listLenientJudge2]))

print('------------------------')


print('------------------------')
print('------------------------')
print('------------------------')
print('------------------------')
print('------------------------')

print('COMBO(REGEX/LSA)(S) vs. Judge 1- F1:', max([(x[0], x[1]) for x in ClistStringentJudge1]))
print('COMBO(REGEX/LSA)(S) vs. Judge 2- F1:', max([(x[0], x[1]) for x in ClistStringentJudge2]))


print('------------------------')
print('COMBO(REGEX/LSA)(I) vs. Judge 1- F1:', max([(x[0], x[1]) for x in ClistIntermediateJudge1]))
print('COMBO(REGEX/LSA)(I) vs. Judge 2- F1:', max([(x[0], x[1]) for x in ClistIntermediateJudge2]))


print('------------------------')
print('COMBO(REGEX/LSA)(L) vs. Judge 1- F1:', max([(x[0], x[1]) for x in ClistLenientJudge1]))
print('COMBO(REGEX/LSA)(L) vs. Judge 2- F1:', max([(x[0], x[1]) for x in ClistLenientJudge2]))

print('------------------------')
